Chain without chat history

In [ ]:
from core.llm.openai.openai_llm import OpenAILLM

llm = OpenAILLM(model="gpt-4o")

In [ ]:
from core.rag.splitter.text_splitter import TextSplitter
from core.rag.vectorstore.chroma import Chroma
from core.rag.document_loaders.dir_loader import DirLoader

docs = DirLoader(file_or_dir="./files/paul_graham_essay.txt").load()
docs = TextSplitter().split_document(docs)
vectorstore = Chroma()
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever(top_k=10)

In [ ]:
from core.rag.document.document import Document
from typing import List


def format_docs(documents: List[Document]):
    return "\n\n".join(doc.text for doc in documents)

In [ ]:
from core.llm.message_parser import StrOutParser
from core.rag.prompts import qa_prompt
from core.flow.flow import identity

rag_flow = (
        {"context": retriever | format_docs, "question": identity}
        | qa_prompt
        | llm
        | StrOutParser()
)

In [ ]:
rag_flow.invoke("what did paul graham do after going to RISD?")

Contextualizing the question

In [ ]:
from core.rag.retrivers.history_aware_retriever import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(llm, retriever)

Chain with chat history

In [ ]:

from core.rag.helpers import create_rag_flow
from core.rag.combine_documents.stuff import create_stuff_documents_flow
from core.prompts.chat_template import ChatTemplate, MessagesPlaceholder

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.
{context}"""
qa_prompt = ChatTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}")
    ]
)

question_answer_flow = create_stuff_documents_flow(llm, qa_prompt)
rag_flow = create_rag_flow(history_aware_retriever, question_answer_flow)

In [ ]:
from core.messages.chat_message import ChatMessage, Role

chat_history = []

question = "what did paul graham do after going to RISD?"
ai_msg_1 = rag_flow.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([ChatMessage(role=Role.USER, content=question), ai_msg_1["answer"]])

second_question = "Where dit he work after that?"
ai_msg_2 = rag_flow.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])